In [1]:
import numpy as np
import os
import sys
import argparse
import glob
import time
sys.path.append("/home/jungmo/apps/caffe/python") # caffe path

import caffe
from shutil import copyfile

GPUID = 1
BIRD_ROOT = "/home/jungmo/Documents/bird/caffe/"

NPY_PATH = os.path.join(BIRD_ROOT,"model","new_32_200", "mean.npy")
CAFFE_BIN = "/home/jungmo/apps/caffe/build/tools/"
network = 'googLeNet'
IMG_SIZE = 224

def binaryproto_to_npy():
    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open( BINARYPROTO_PATH , 'rb' ).read()
    blob.ParseFromString(data)
    arr = np.array( caffe.io.blobproto_to_array(blob) )
    out = arr[0]
    np.save( NPY_PATH, out )

def evaluate_model(dst):
        input_file = os.path.join(TEST)
        output_file = os.path.join(BIRD_ROOT, "db/output.npy")
        model_def = os.path.join(BIRD_ROOT, "model_prototxt/"+network+"/deploy.prototxt")
        pretrained_model = os.path.join(BIRD_ROOT, "model/new_32_200/googlenet_iter_60000.caffemodel")
        gpu = True
        images_dim='224,224'
        mean_file = NPY_PATH
        ext='bmp'
        channel_swap='2,1,0'
        raw_scale = 255.0
        center_only=True
        input_scale = None
        image_dims = [int(s) for s in images_dim.split(',')]

        mean, channel_swap = None, None
        if mean_file:
            mean = np.load(mean_file)
        if channel_swap:
            channel_swap=[int(s) for s in args.channel_swap.split(',')]
        caffe.set_mode_gpu()
        #caffe.set_device(GPUID)
        # Make classifier.
        classifier = caffe.Classifier(model_def, pretrained_model,
                image_dims=image_dims, mean=mean,
                input_scale=input_scale, raw_scale=raw_scale,
                channel_swap=channel_swap)

        # Load numpy array (.npy), directory glob (*.jpg), or image file.
        imgs_list = tuple(open(input_file,'r'))
        print("Loading file: %s" % input_file)
        inputs =[caffe.io.load_image(im_f)
                     for im_f in glob.glob(input_file + '/*.' + ext)]

        print("Classifying %d inputs." % len(imgs_list))

        BATCH_SIZE = 100

        # Classify.
        confusion_matrix = np.zeros((4,4),dtype=np.int32)
        accuracy = 0.0
        start = time.time()
        for i in range(0, len(imgs_list), BATCH_SIZE):
            if i > 0 and i % (BATCH_SIZE * 25) == 0:
                print("Finished %d imgs." % (i))
                cur_accuracy = accuracy / i * 100
                print("Accuracy: %.2f " % cur_accuracy)
                print("Confusion Matrix:")
                print confusion_matrix
            imgs = [imgs_list[i + j].split(' ')[0] for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            gts = [int(imgs_list[i + j].split(' ')[1]) for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            inputs = [caffe.io.load_image(path) for path in imgs]
            predictions = classifier.predict(inputs, not center_only)
            for j in range(len(imgs)):
                probs = predictions[j]
                pred = np.argmax(probs)
                confusion_matrix[gts[j]][pred] += 1
                if pred == gts[j]:
                    accuracy += 1.0

        accuracy = accuracy / len(imgs_list) * 100

        print("Done in %.2f s." % (time.time() - start))
        # Save
        print("Accuracy: %.2f " % accuracy)
        print("Confusion Matrix:")
        print confusion_matrix
        np.save(dst, confusion_matrix)

In [2]:

'''    
print "Make DB : Training set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE)+ " / " + TRAIN + " " + BIRD_ROOT + "db/train_imageData_lmDB")

print "Make DB : Test set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE) + " / " + TEST + " " + BIRD_ROOT + "db/val_imageData_lmDB")

print "Make image mean file"
os.system(os.path.join(CAFFE_BIN,"compute_image_mean.bin")+
               " -backend=lmdb "+ BIRD_ROOT + "db/train_imageData_lmDB " + BIRD_ROOT+ "db/mean.binaryproto")
print "Image mean file .binaryproto to .npy"
binaryproto_to_npy()   

print "Training model"
os.system(os.path.join(CAFFE_BIN,"caffe.bin")+" train "
               "-solver="+BIRD_ROOT+"model/"+network+"/solver.prototxt "+" -weights="+BIRD_ROOT+"model/"+network+"/bvlc_googlenet.caffemodel "+ "-gpu all")
'''
print "Evaluate model"
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_200.txt"
evaluate_model("test_32_200.npy")


Evaluate model
Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_200.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 95.28 
Confusion Matrix:
[[2382   26   36   56]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 95.78 
Confusion Matrix:
[[4789   57   61   93]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 95.60 
Confusion Matrix:
[[6448   77   84  129]
 [  20  722    8   12]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 95.91 
Confusion Matrix:
[[8609   80  115  165]
 [  24  982   11   14]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 95.50 
Confusion Matrix:
[[9564   82  134  187]
 [  24  982   11   14]
 [  11   16 1392   83]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 95.13 
Confusion Matrix:
[[9564   82  134  187]
 [  24  982   11   14]
 [  32   54 3724  192]
 [   0   

In [3]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_150.txt"
evaluate_model("test_32_150.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_150.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.60 
Confusion Matrix:
[[2440   12   17   31]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.38 
Confusion Matrix:
[[4869   27   52   52]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.99 
Confusion Matrix:
[[6560   36   77   65]
 [  25  714   10   13]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.99 
Confusion Matrix:
[[8741   38   93   97]
 [  36  958   16   21]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.94 
Confusion Matrix:
[[9721   39   96  111]
 [  36  958   16   21]
 [  21    7 1439   35]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 96.70 
Confusion Matrix:
[[9721   39   96  111]
 [  36  958   16   21]
 [  65   17 3826   94]
 [   0    0    0    0]]


In [4]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_140.txt"
evaluate_model("test_32_140.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_140.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.04 
Confusion Matrix:
[[2426   13   32   29]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.32 
Confusion Matrix:
[[4866   24   57   53]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.81 
Confusion Matrix:
[[6551   31   83   73]
 [  26  710   15   11]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.73 
Confusion Matrix:
[[8720   34  103  112]
 [  39  953   20   19]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.78 
Confusion Matrix:
[[9696   34  113  124]
 [  39  953   20   19]
 [  26    2 1448   26]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 96.68 
Confusion Matrix:
[[9696   34  113  124]
 [  39  953   20   19]
 [  65    6 3853   78]
 [   0    0    0    0]]


In [ ]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_130.txt"
evaluate_model("test_32_130.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_130.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 96.80 
Confusion Matrix:
[[2420   13   34   33]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 96.98 
Confusion Matrix:
[[4849   22   73   56]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.51 
Confusion Matrix:
[[6540   31   90   77]
 [  38  698   15   11]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.43 
Confusion Matrix:
[[8700   35  119  115]
 [  49  943   18   21]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.50 
Confusion Matrix:
[[9658   35  135  139]
 [  49  943   18   21]
 [  17    3 1462   20]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 96.51 
Confusion Matrix:
[[9658   35  135  139]
 [  49  943   18   21]
 [  45    8 3875   74]
 [   0    0    0    0]]


In [11]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_120.txt"
evaluate_model("test_32_120.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_120.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 96.60 
Confusion Matrix:
[[2415    9   34   42]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 96.76 
Confusion Matrix:
[[4838   15   70   77]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.08 
Confusion Matrix:
[[6517   21   98  102]
 [  44  689   11   18]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 95.77 
Confusion Matrix:
[[8647   24  148  150]
 [  55  930   16   30]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 95.94 
Confusion Matrix:
[[9606   24  169  168]
 [  55  930   16   30]
 [  27    2 1457   16]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 96.25 
Confusion Matrix:
[[9606   24  169  168]
 [  55  930   16   30]
 [  61    3 3901   37]
 [   0    0    0    0]]


In [9]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_110.txt"
evaluate_model("test_32_110.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_110.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.00 
Confusion Matrix:
[[2425   11   27   37]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.26 
Confusion Matrix:
[[4863   19   48   70]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.41 
Confusion Matrix:
[[6563   24   61   90]
 [  66  668    0   28]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.33 
Confusion Matrix:
[[8728   25   74  142]
 [  91  905    0   35]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 95.67 
Confusion Matrix:
[[9706   26   77  158]
 [  91  905    0   35]
 [  61    0 1348   93]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 94.55 
Confusion Matrix:
[[9706   26   77  158]
 [  91  905    0   35]
 [ 170    2 3571  259]
 [   0    0    0    0]]


In [10]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_100.txt"
evaluate_model("test_32_100.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_32_100.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.72 
Confusion Matrix:
[[2443    8   16   33]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.04 
Confusion Matrix:
[[4902   15   32   51]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.87 
Confusion Matrix:
[[6609   20   42   67]
 [  75  656    0   31]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.96 
Confusion Matrix:
[[8806   20   42  101]
 [ 101  890    0   40]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 93.56 
Confusion Matrix:
[[9793   21   42  111]
 [ 101  890    0   40]
 [ 147    4 1012  339]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 88.58 
Confusion Matrix:
[[9793   21   42  111]
 [ 101  890    0   40]
 [ 461   11 2604  926]
 [   0    0    0    0]]
